<a href="https://colab.research.google.com/github/AmiJoba22/RateMD-Datapipe/blob/main/pythonscripts%20/%20extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Get data from source to put in destination
#Destination has an address; source has an address
!pip install azure-storage-blob
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.4/207.4 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pymongo
from pymongo import MongoClient
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [3]:
# read the config file that is json
import json
with open('config.json') as config_file:
  config = json.load(config_file)

#read information about the destination
DESTINATION_CONNECTION_STRING = config['DESTINATION_CONNECTION_STRING']
DESTINATION_CONTAINER_NAME = config['DESTINATION_CONTAINER_NAME']
DESTINATION_FILE_NAME = config ['DESTINATION_FILE_NAME']

# read information about mongodb
MONGODB_CONNECTION_STRING = config ['MONGODB_CONNECTION_STRING']
MONGO_DB_NAME = config ['MONGO_DB_NAME']
MONGO_COLLECTION_NAME = config ['MONGO_COLLECTION_NAME']
print(MONGO_COLLECTION_NAME)

ratemd


In [4]:
# connect to mongodb and test the connection using ping
client = MongoClient(MONGODB_CONNECTION_STRING)
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
db = client[MONGO_DB_NAME]
collection = db[MONGO_COLLECTION_NAME]
print(collection)

Pinged your deployment. You successfully connected to MongoDB!
Collection(Database(MongoClient(host=['20.232.135.212:27017'], document_class=dict, tz_aware=False, connect=True, authmechanism='DEFAULT', authsource='admin'), 'healthrate'), 'ratemd')


In [6]:
from azure.storage.blob import BlobServiceClient
import pandas as pd

CHUNK_SIZE = 100000
total_docs = collection.count_documents({})
print(f"Total documents in collection: {total_docs}")

blob_service_client = BlobServiceClient.from_connection_string(DESTINATION_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(DESTINATION_CONTAINER_NAME)

# Create the container if it does not exist
try:
    container_client.create_container()
    print(f"Container '{DESTINATION_CONTAINER_NAME}' created.")
except Exception as e:
    # Handle exceptions, e.g., if the container already exists
    print(f"Could not create container '{DESTINATION_CONTAINER_NAME}': {e}")


for skip in range(0, total_docs, CHUNK_SIZE):
    chunk_count = (skip // CHUNK_SIZE) + 1
    blob_chunk_name = f"{DESTINATION_FILE_NAME}_chunk_{chunk_count}.csv"

    # 🛑 Skip if blob already exists
    if container_client.get_blob_client(blob_chunk_name).exists():
        print(f"⏭️ Chunk {chunk_count} already exists in Azure. Skipping...")
        continue

    print(f"🚧 Processing chunk {chunk_count}")

    # Fetch a chunk of documents
    cursor = collection.find().skip(skip).limit(CHUNK_SIZE)
    chunk_data = list(cursor)

    # Convert to DataFrame
    df_chunk = pd.json_normalize(chunk_data)

    # Save to CSV
    local_chunk_file_name = f"mongo_collection_data_chunk_{chunk_count}.csv"
    df_chunk.to_csv(local_chunk_file_name, index=False)

    # Upload to Azure
    with open(local_chunk_file_name, "rb") as data:
        container_client.upload_blob(name=blob_chunk_name, data=data, overwrite=True)

    print(f"✅ Chunk {chunk_count} uploaded as '{blob_chunk_name}'")

Total documents in collection: 2204490
Could not create container 'ratemd': The specified container already exists.
RequestId:34afabab-f01e-006a-5ea8-eb5db4000000
Time:2025-07-02T23:25:13.6840762Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:34afabab-f01e-006a-5ea8-eb5db4000000
Time:2025-07-02T23:25:13.6840762Z</Message></Error>
🚧 Processing chunk 1
✅ Chunk 1 uploaded as '_chunk_1.csv'
🚧 Processing chunk 2
✅ Chunk 2 uploaded as '_chunk_2.csv'
🚧 Processing chunk 3
✅ Chunk 3 uploaded as '_chunk_3.csv'
🚧 Processing chunk 4
✅ Chunk 4 uploaded as '_chunk_4.csv'
🚧 Processing chunk 5
✅ Chunk 5 uploaded as '_chunk_5.csv'
🚧 Processing chunk 6
✅ Chunk 6 uploaded as '_chunk_6.csv'
🚧 Processing chunk 7
✅ Chunk 7 uploaded as '_chunk_7.csv'
🚧 Processing chunk 8
✅ Chunk 8 uploaded as '_chunk_8.csv'
🚧 Processing chunk 9
✅ Chunk 9 uploaded as '_chunk_9.csv'
🚧 Processin